### 지훈이의 타이타닉 모델 만들기 대작전!- 데이터셋 보기, 대략적으로


In [86]:
import pandas as pd
import numpy as np
import copy

In [87]:
df_train_raw = pd.read_csv("train.csv")
df_test_raw = pd.read_csv("test.csv")

# 파일에서 읽어온 값들을 깊은 복사로 새로 객체를 만들어서 문제가 생기면 코드를 위의 원본데이터를 재활용한다.
df_train = copy.copy(df_train_raw)
df_test = copy.copy(df_test_raw)

In [88]:
df_train.head().values

array([[1, 0, 3, 'Braund, Mr. Owen Harris', 'male', 22.0, 1, 0,
        'A/5 21171', 7.25, nan, 'S'],
       [2, 1, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
        'female', 38.0, 1, 0, 'PC 17599', 71.2833, 'C85', 'C'],
       [3, 1, 3, 'Heikkinen, Miss. Laina', 'female', 26.0, 0, 0,
        'STON/O2. 3101282', 7.925, nan, 'S'],
       [4, 1, 1, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
        'female', 35.0, 1, 0, '113803', 53.1, 'C123', 'S'],
       [5, 0, 3, 'Allen, Mr. William Henry', 'male', 35.0, 0, 0,
        '373450', 8.05, nan, 'S']], dtype=object)

In [89]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [90]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [91]:
print("---train set data---")
print(df_train.isnull().sum())
print("데이터 shape :", end='')
print(df_train.shape)
print()


---train set data---
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
데이터 shape :(891, 12)



In [92]:
print("---test set data---")
print(df_test.isnull().sum())
print("데이터 shape :", end='')
print(df_test.shape)
print()

---test set data---
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
데이터 shape :(418, 11)



### 데이터 전처리 - 테스트데이터는 행 만지지마라

In [93]:
# 'Age', 'Cabin' 열 제거 
df_train = df_train.drop(columns=[ 'Age', 'Cabin'])
df_test = df_test.drop(columns=[ 'Age', 'Cabin'])

너네들이 전략을 세울거 아니야 아 일단 알아서 해 자 그러니까 그냥 마셔라
제일 먼저 상관관계 큰거 찾기- 수치형 모델 선택
최소한 0.5 이상이 1개는 있어야 된다

등급에 대한 상관관계 있음

In [94]:
# 'Embarked' 결측치를 최빈값으로 채우기
most_common_embarked = df_train['Embarked'].mode()[0]
df_train['Embarked'] = df_train['Embarked'].fillna(most_common_embarked)


In [95]:
# 'Fare' 결측치를 중앙값으로 채우기
median_fare = df_test['Fare'].median()
df_test['Fare'] = df_test['Fare'].fillna(median_fare)


In [96]:
print("---train set data (after filling missing values)---")
print(df_train.isnull().sum())
print("---test set data (after filling missing values)---")
print(df_test.isnull().sum())


---train set data (after filling missing values)---
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
---test set data (after filling missing values)---
PassengerId    0
Pclass         0
Name           0
Sex            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


### 인코딩하기- 데이터 전처리의 꽃

In [97]:
from sklearn.preprocessing import LabelEncoder

# 'Name'에서 호칭 추출 함수
def extract_title(name):
    return name.split(',')[1].split('.')[0].strip()

# 'Name'에서 호칭 추출
df_train['Title'] = df_train['Name'].apply(extract_title)
df_test['Title'] = df_test['Name'].apply(extract_title)

# 희귀 호칭을 'Rare'로 묶기
def replace_rare_titles(df, threshold=5):
    title_counts = df['Title'].value_counts()
    rare_titles = title_counts[title_counts <= threshold].index
    df['Title'] = df['Title'].replace(rare_titles, 'Rare')

replace_rare_titles(df_train)
replace_rare_titles(df_test)

# 라벨 인코딩
label_encoder = LabelEncoder()
df_train['Title'] = label_encoder.fit_transform(df_train['Title'])
df_test['Title'] = label_encoder.transform(df_test['Title'])

# 결과 확인
print(df_train[['Name', 'Title']].head())
print(df_test[['Name', 'Title']].head())


                                                Name  Title
0                            Braund, Mr. Owen Harris      3
1  Cumings, Mrs. John Bradley (Florence Briggs Th...      4
2                             Heikkinen, Miss. Laina      2
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)      4
4                           Allen, Mr. William Henry      3
                                           Name  Title
0                              Kelly, Mr. James      3
1              Wilkes, Mrs. James (Ellen Needs)      4
2                     Myles, Mr. Thomas Francis      3
3                              Wirz, Mr. Albert      3
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)      4


In [98]:
# 'Sex'와 'Embarked' 원핫 인코딩
df_train = pd.get_dummies(df_train, columns=['Sex', 'Embarked'], drop_first=True)
df_test = pd.get_dummies(df_test, columns=['Sex', 'Embarked'], drop_first=True)

# 'Name', 'Ticket', 'Cabin' 열 삭제
df_train = df_train.drop(columns=['Name', 'Ticket'])
df_test = df_test.drop(columns=['Name', 'Ticket'])

# 결과 확인
print(df_train.head())
print(df_test.head())


   PassengerId  Survived  Pclass  SibSp  Parch     Fare  Title  Sex_male  \
0            1         0       3      1      0   7.2500      3      True   
1            2         1       1      1      0  71.2833      4     False   
2            3         1       3      0      0   7.9250      2     False   
3            4         1       1      1      0  53.1000      4     False   
4            5         0       3      0      0   8.0500      3      True   

   Embarked_Q  Embarked_S  
0       False        True  
1       False       False  
2       False        True  
3       False        True  
4       False        True  
   PassengerId  Pclass  SibSp  Parch     Fare  Title  Sex_male  Embarked_Q  \
0          892       3      0      0   7.8292      3      True        True   
1          893       3      1      0   7.0000      4     False       False   
2          894       2      0      0   9.6875      3      True        True   
3          895       3      0      0   8.6625      3      True 

In [99]:
df_train.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,Title,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,1,0,7.2500,3,True,False,True
1,2,1,1,1,0,71.2833,4,False,False,False
2,3,1,3,0,0,7.9250,2,False,False,True
3,4,1,1,1,0,53.1000,4,False,False,True
4,5,0,3,0,0,8.0500,3,True,False,True


In [100]:
# Survived와 다른 변수들 간의 상관관계 계산
correlation_with_survived = df_train.corr()['Survived'].sort_values(ascending=False)

# 상관관계 표 출력
print("---Correlation with Survived---")
print(correlation_with_survived)


---Correlation with Survived---
Survived       1.000000
Fare           0.257307
Parch          0.081629
Embarked_Q     0.003650
PassengerId   -0.005007
SibSp         -0.035322
Title         -0.060604
Embarked_S    -0.149683
Pclass        -0.338481
Sex_male      -0.543351
Name: Survived, dtype: float64


In [101]:
df_train.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,Title,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,1,0,7.2500,3,True,False,True
1,2,1,1,1,0,71.2833,4,False,False,False
2,3,1,3,0,0,7.9250,2,False,False,True
3,4,1,1,1,0,53.1000,4,False,False,True
4,5,0,3,0,0,8.0500,3,True,False,True


## 지피티가 생각하는 모델- 로지스틱 회귀

In [38]:
# 필요한 라이브러리 임포트
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 특성과 타겟 변수 분리
X = df_train[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 중요한 변수들만 선택
y = df_train['Survived']  # 타겟 변수

# 훈련 데이터와 테스트 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 특성 데이터 표준화 (로지스틱 회귀는 스케일링에 민감함)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 로지스틱 회귀 모델 훈련
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# 예측 및 평가
y_pred = model.predict(X_val_scaled)

# 정확도 평가
accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)

# 결과 출력
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Accuracy: 0.7654
Confusion Matrix:
[[80 25]
 [17 57]]
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.76      0.79       105
           1       0.70      0.77      0.73        74

    accuracy                           0.77       179
   macro avg       0.76      0.77      0.76       179
weighted avg       0.77      0.77      0.77       179



In [37]:
# 테스트 데이터에 대해 동일한 전처리 수행 (스케일링)
X_test = df_test[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 테스트 데이터의 특성 선택
X_test_scaled = scaler.transform(X_test)  # 스케일링 적용

# 테스트 데이터에 대한 생존 여부 예측
test_predictions = model.predict(X_test_scaled)

# 결과를 DataFrame으로 생성
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],  # 승객 번호
    'Survived': test_predictions            # 예측된 생존 여부
})

# CSV 파일로 저장
submission.to_csv('submission.csv', index=False)

print("submission.csv 파일이 생성되었습니다.")


submission.csv 파일이 생성되었습니다.


## 결정 트리 모델


In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 훈련 데이터 준비
X_train = df_train[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 훈련 데이터의 특성 선택
y_train = df_train['Survived']  # 타겟 변수

# 테스트 데이터 준비
X_test = df_test[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 테스트 데이터의 특성 선택

# 결정 트리 모델 생성 및 학습
dt_model = DecisionTreeClassifier(random_state=42)  # 결정 트리 모델 생성
dt_model.fit(X_train, y_train)  # 모델 학습

# 훈련 데이터에 대한 예측
train_predictions = dt_model.predict(X_train)

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"훈련 데이터 정확도: {train_accuracy:.4f}")

# 테스트 데이터 예측
test_predictions = dt_model.predict(X_test)




훈련 데이터 정확도: 0.9091


In [46]:
# 결과를 DataFrame으로 생성
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],  # 승객 번호
    'Survived': test_predictions            # 예측된 생존 여부
})

# CSV 파일로 저장
submission.to_csv('submission_decision_tree.csv', index=False)

print("submission_decision_tree.csv 파일이 생성되었습니다.")


submission_decision_tree.csv 파일이 생성되었습니다.


## 랜덤포레스트

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 훈련 데이터 준비
X_train = df_train[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 훈련 데이터의 특성 선택
y_train = df_train['Survived']  # 타겟 변수

# 테스트 데이터 준비
X_test = df_test[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 테스트 데이터의 특성 선택

# 랜덤 포레스트 모델 생성 및 학습
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)  # 랜덤 포레스트 모델 생성
rf_model.fit(X_train, y_train)  # 모델 학습

# 훈련 데이터에 대한 예측
train_predictions = rf_model.predict(X_train)

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"랜덤 포레스트 - 훈련 데이터 정확도: {train_accuracy:.4f}")

# 테스트 데이터 예측
test_predictions = rf_model.predict(X_test)




랜덤 포레스트 - 훈련 데이터 정확도: 0.9091


In [66]:
# 결과를 DataFrame으로 생성
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],  # 승객 번호
    'Survived': test_predictions            # 예측된 생존 여부
})

# CSV 파일로 저장
submission.to_csv('submission_random_forest.csv', index=False)

print("submission_random_forest.csv 파일이 생성되었습니다.")

submission_random_forest.csv 파일이 생성되었습니다.


## 로지스틱 회귀 리트

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 훈련 데이터 준비
X_train = df_train[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 훈련 데이터의 특성 선택
y_train = df_train['Survived']  # 타겟 변수

# 테스트 데이터 준비
X_test = df_test[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 테스트 데이터의 특성 선택

# 로지스틱 회귀 모델 생성 및 학습
logistic_model = LogisticRegression(random_state=42, max_iter=1000)  # 로지스틱 회귀 모델 생성
logistic_model.fit(X_train, y_train)  # 모델 학습

# 훈련 데이터에 대한 예측
train_predictions = logistic_model.predict(X_train)

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"로지스틱 회귀 - 훈련 데이터 정확도: {train_accuracy:.4f}")

# 테스트 데이터 예측
test_predictions = logistic_model.predict(X_test)




로지스틱 회귀 - 훈련 데이터 정확도: 0.7778


In [85]:
# 결과를 DataFrame으로 생성
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],  # 승객 번호
    'Survived': test_predictions            # 예측된 생존 여부
})

# CSV 파일로 저장
submission.to_csv('submission_logistic_regression.csv', index=False)

print("submission_logistic_regression.csv 파일이 생성되었습니다.")

submission_logistic_regression.csv 파일이 생성되었습니다.


## KNN

In [103]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 훈련 데이터 준비
X_train = df_train[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 훈련 데이터의 특성 선택
y_train = df_train['Survived']  # 타겟 변수

# 테스트 데이터 준비
X_test = df_test[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]  # 테스트 데이터의 특성 선택

# KNN 모델 생성 및 학습
knn_model = KNeighborsClassifier(n_neighbors=5)  # K=5로 설정
knn_model.fit(X_train, y_train)

# 훈련 데이터에 대한 예측
train_predictions = knn_model.predict(X_train)

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"KNN - 훈련 데이터 정확도: {train_accuracy:.4f}")

# 테스트 데이터 예측
test_predictions = knn_model.predict(X_test)




KNN - 훈련 데이터 정확도: 0.8395


In [104]:
# 결과를 DataFrame으로 생성
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],  # 승객 번호
    'Survived': test_predictions            # 예측된 생존 여부
})

# CSV 파일로 저장
submission.to_csv('submission_knn.csv', index=False)

print("submission_knn.csv 파일이 생성되었습니다.")

submission_knn.csv 파일이 생성되었습니다.


## 앙상블학습 - 결정트리 + 랜덤포레스트

In [105]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# 훈련 데이터 준비
X_train = df_train[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]
y_train = df_train['Survived']

# 테스트 데이터 준비
X_test = df_test[['Pclass', 'Fare', 'Sex_male', 'Embarked_S', 'Embarked_Q']]

# 랜덤 포레스트 모델
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 결정 트리 모델
dt_model = DecisionTreeClassifier(random_state=42)

# VotingClassifier로 앙상블 모델 생성
ensemble_model = VotingClassifier(
    estimators=[
        ('random_forest', rf_model),
        ('decision_tree', dt_model)
    ],
    voting='soft'  # 'soft'는 확률 기반 투표, 'hard'는 단순 투표
)

# 모델 학습
ensemble_model.fit(X_train, y_train)

# 훈련 데이터에 대한 예측
train_predictions = ensemble_model.predict(X_train)

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"앙상블 모델 - 훈련 데이터 정확도: {train_accuracy:.4f}")

# 테스트 데이터 예측
test_predictions = ensemble_model.predict(X_test)



앙상블 모델 - 훈련 데이터 정확도: 0.9091


In [106]:
# 결과를 DataFrame으로 생성
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],
    'Survived': test_predictions
})

# CSV 파일로 저장
submission.to_csv('submission_ensemble.csv', index=False)

print("submission_ensemble.csv 파일이 생성되었습니다.")


submission_ensemble.csv 파일이 생성되었습니다.


## 앙상블학습- 이때까지 만든거 다 짬뽕

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 훈련 데이터 준비
X_train = df_train[['Pclass', 'Fare', 'Sex_male', 'Embarked_C', 'Embarked_Q']]
y_train = df_train['Survived']

# 테스트 데이터 준비
X_test = df_test[['Pclass', 'Fare', 'Sex_male', 'Embarked_C', 'Embarked_Q']]

# 모델 정의
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # 랜덤 포레스트
dt_model = DecisionTreeClassifier(random_state=42)                   # 결정 트리
lr_model = LogisticRegression(max_iter=1000, random_state=42)        # 로지스틱 회귀
knn_model = KNeighborsClassifier(n_neighbors=5)                      # KNN

# VotingClassifier로 앙상블 모델 생성
ensemble_model = VotingClassifier(
    estimators=[
        ('random_forest', rf_model),
        ('decision_tree', dt_model),
        ('logistic_regression', lr_model),
        ('knn', knn_model)
    ],
    voting='soft'  # 'soft'는 확률 기반 투표, 'hard'는 단순 투표
)

# 모델 학습
ensemble_model.fit(X_train, y_train)

# 훈련 데이터에 대한 예측
train_predictions = ensemble_model.predict(X_train)

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"앙상블 모델 - 훈련 데이터 정확도: {train_accuracy:.4f}")

# 테스트 데이터 예측
test_predictions = ensemble_model.predict(X_test)



In [ ]:
# 결과를 DataFrame으로 생성
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],
    'Survived': test_predictions
})

# CSV 파일로 저장
submission.to_csv('submission_ensemble_all.csv', index=False)

print("submission_ensemble_all.csv 파일이 생성되었습니다.")
